# Data Cleaning and Preparation

During the course of doing data analysis and modeling, a significant amount of time is spent on data preparation: loading, cleaning, transforming, and rearranging. Such tasks are often reported to take up 80% of more of an analyst's time. Fortunately, pandas, along with built-in Python language features, provides you with a high-level, flexible, and fast set of tools to enable you to manipulate data into the right form. 

## Handling Missing Data

Missing data occurs commonly in many data analysis applications. One of the goals of pandas is to make working with missing data as painless as possible. The way that missing data is represented in pandas objects is somewhat imperfect, but it is functional for a lot of users. For numeric data, pandas uses the floating-point value `NaN` (Not a Number) to represent missing data:

In [1]:
import pandas as pd
import numpy as np

In [2]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

Detecting null values:

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In pandas, we reference missing data as NA, which stands for *not available*. In statistics applications, NA data may either be data that does not exist or that exists but was not observed (through problems with data collection for example). When cleaning up data for analysis, it is often important to do analysis on the missing data itself to identify data collection problems or potential biases in the data caused by missing data.

The built-in Python `None` value is also treated as NA in object arrays:

In [4]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

See table below for a list of some functions related to missing data handling:

**Argument** | **Description**
--- | ---
`dropna` | Filter axis labels based on whether values for each label have missing data, with varying thresholds for how much missing data to tolerate.
`fillna` | Fill in missing data with some value or using an interpolation method such as 'ffill' or 'bfill'.
`isnull` | Return boolean values indicating which values are missing/NA.
`notnull` | Negation of `isnull`.

### Filtering Out Missing Data

There are a few ways to filter out missing data. While you always have the option to do it by hand using `pandas.isnull` and boolean indexing, the `dropna` can be helpful. On a Series, it returns the Series with only the non-null data and index values:

In [5]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

This is equivalent to:

In [6]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

Handling missing data in DataFrame objects can be a bit more complex. You may want to drop rows or columns that are all NA or only those containing any NAs. `dropna` by default, drops any row containing a missing value:

In [7]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], 
                     [NA, NA, NA], [NA, 6.5, 3.]])

cleaned = data.dropna()
print("data:\n{}\n".format(data))
cleaned

data:
     0    1    2
0  1.0  6.5  3.0
1  1.0  NaN  NaN
2  NaN  NaN  NaN
3  NaN  6.5  3.0



,0,1,2
0,1.0,6.5,3.0


Passing `how='all'` argument will only drop rows that are all NA:

In [8]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop columns in the same way, pass `axis=1`:

In [9]:
data[4] = NA
print("data:\n{}\n".format(data))

data.dropna(axis=1, how='all')

data:
     0    1    2   4
0  1.0  6.5  3.0 NaN
1  1.0  NaN  NaN NaN
2  NaN  NaN  NaN NaN
3  NaN  6.5  3.0 NaN



,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Another useful option is `thresh`, which is useful for dealing with time series data. Suppose you want to keep only rows containing a certain number of observations. You can indicate this with the `thresh` argument:

In [10]:
df = pd.DataFrame(np.random.randn(7, 3))

# imputing NA values
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA

print("{}\n".format(df))

df.dropna(thresh=2)

          0         1         2
0 -0.982260       NaN       NaN
1  0.454544       NaN       NaN
2  1.602384       NaN -1.943291
3 -1.231869       NaN -1.063765
4 -0.473087 -1.150014 -1.070185
5 -0.610113 -0.569656  1.065850
6 -1.137286  0.344105 -0.509579



,0,1,2
2,1.602384,NaN,-1.943291
3,-1.231869,NaN,-1.063765
4,-0.473087,-1.150014,-1.070185
5,-0.610113,-0.569656,1.065850
6,-1.137286,0.344105,-0.509579


### Filling in Missing Data

Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the "holes" in any number of ways. For most purposes, the `fillna` method is the workhorse function to use. Calling `fillna` with a constant replaces missing values with that value:

In [11]:
df.fillna(0)

,0,1,2
0,-0.982260,0.000000,0.000000
1,0.454544,0.000000,0.000000
2,1.602384,0.000000,-1.943291
3,-1.231869,0.000000,-1.063765
4,-0.473087,-1.150014,-1.070185
5,-0.610113,-0.569656,1.065850
6,-1.137286,0.344105,-0.509579


Calling `fillna` with a dict, you can use a different fill value for each column:

In [12]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.982260,0.500000,0.000000
1,0.454544,0.500000,0.000000
2,1.602384,0.500000,-1.943291
3,-1.231869,0.500000,-1.063765
4,-0.473087,-1.150014,-1.070185
5,-0.610113,-0.569656,1.065850
6,-1.137286,0.344105,-0.509579


`fillna()` method returns a new object, but you can modify the existing object in-place:

In [13]:
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,-0.982260,0.000000,0.000000
1,0.454544,0.000000,0.000000
2,1.602384,0.000000,-1.943291
3,-1.231869,0.000000,-1.063765
4,-0.473087,-1.150014,-1.070185
5,-0.610113,-0.569656,1.065850
6,-1.137286,0.344105,-0.509579


The same interpolation methods available for reindexing can be used with `fillna`:

In [14]:
df = pd.DataFrame(np.random.randn(6,3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
print("{}\n".format(df))

df.fillna(method='ffill')

          0         1         2
0 -1.345421 -0.332222  0.702045
1  0.405551 -0.160960  0.296455
2 -0.105774       NaN -0.156298
3  0.121257       NaN  1.115117
4  0.066063       NaN       NaN
5  0.622295       NaN       NaN



,0,1,2
0,-1.345421,-0.332222,0.702045
1,0.405551,-0.160960,0.296455
2,-0.105774,-0.160960,-0.156298
3,0.121257,-0.160960,1.115117
4,0.066063,-0.160960,1.115117
5,0.622295,-0.160960,1.115117


In [15]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-1.345421,-0.332222,0.702045
1,0.405551,-0.160960,0.296455
2,-0.105774,-0.160960,-0.156298
3,0.121257,-0.160960,1.115117
4,0.066063,NaN,1.115117
5,0.622295,NaN,1.115117


You might also pass the mean or median value of a Series with `fillna()`:

In [16]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

Table below summarises function arguments for `fillna`:

**Argument** | **Description**
--- | ---
`value` | Scalar value or dict-like object to use to fill missing values
`method` | Interpolation; by default `ffill` if function called with no other arguments
`axis` | Axis to fill on; default `axis=0`
`inplace` | Modify the calling object without producing a copy
`limit` | For forward and backward filling, maximum number of consecutive periods to fill

## Data Transformation

Filtering, cleaning and other transformation are another class of important operations when dealing with data for analysis:

### Removing Duplicates

Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [17]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method `duplicated` returns a boolean Series indicating whether each row is a duplicate (has been observed in a previous row) or not:

In [18]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relatedly, `drop_duplicates` return a DataFrame where the `duplicated` array is `False`:

In [19]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Alternatively, you can specify any subset of the data to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicates only based on the 'k1' column:

In [20]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated` and `drop_duplicates` by default keep the first observed value combination. Passing `keep='last'` will return the last one:

In [21]:
data.drop_duplicates(['k1','k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. Consider the following hypothetical data collected about various kinds of meat:

In [22]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 
                              'Pastrami', 'corned beef', 'Bacon', 
                              'pastrami', 'honey ham', 'nova lox'],
                    'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose we want to add a column indicating the type of animal that each food came from. We can achieve this using the `map` method - a convenient way to perform element-wise transformations and other data cleaning-related operations. The `map` method on a Series accepts a function or dict-like object containing a mapping. 

But first, we'll need a mapping of each distinct meat type to the kind of animal. We also need to convert each value in `food` column to lowercase using the `str.lower` method before applying the `map` method:

In [23]:
# initialise dict for mapping later on
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

# convert all food labels to lowercase before applying mapping
lowercased = data['food'].str.lower()

# append 'animal' column based on mapping earlier
data['animal']  = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


We could pass a function that does all the work using `lambda` function:

In [24]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing Values

Filling in missing data with the `fillna` method is a special case of more general value replacement. As you've already seen, `map` can be used to modify a subset of values in an object but `replace` method provides a simpler and more flexible way to do so. Let's consider this Series:

In [25]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [26]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [27]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, we can pass a list of substitutes:

In [28]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The argument passed can also be a dict:

In [29]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects. You can also modify the axes in-place without creating a new data structures. Here's a simple example:

In [30]:
data = pd.DataFrame(np.arange(12).reshape((3,4)),
                   index=['Ohio', 'Colorado', 'New York'],
                   columns=['one', 'two', 'three', 'four'])

transform = lambda x: x[:4].upper()

data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


You can also use the `rename` method if you want to create a transformed version of a dataset without modifying the original:

In [31]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


Notably, `rename` can also be used in conjunction with a dict-like object providing new values for a subset of the axis labels:

In [32]:
data.rename(index={'OHIO': 'INDIANA'},
           columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


Should you wish to modify a dataset in-place, pass `inplace=True`:

In [33]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretisation and Binning

Continuous data is often discretised or otherwise separated into "bins" for analysis. Suppose we have data about a group of people in a study, and we want to group them into discrete age buckets. We can divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To do this, we have to use `cut` function in pandas:

In [34]:
ages = [20, 22, 25, 27, 21, 23, 37, 61, 45, 41, 32]
bins = [17, 25, 35, 60, 100]

cats = pd.cut(ages, bins)
cats

[(17, 25], (17, 25], (17, 25], (25, 35], (17, 25], ..., (35, 60], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 11
Categories (4, interval[int64]): [(17, 25] < (25, 35] < (35, 60] < (60, 100]]

The object returned is a special `Categorical` object. The output you see describes the bins computed by `pandas.cut`. 

You can treat it like an array of strings indicating the bin name; internally it contains a `categories` array specifying the distinct category names along with a labelling for the `ages` data in the `codes` attribute:

In [35]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 3, 2, 2, 1], dtype=int8)

In [36]:
cats.categories

IntervalIndex([(17, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [37]:
# doing a value count for cats array
pd.value_counts(cats)

(17, 25]     5
(35, 60]     3
(25, 35]     2
(60, 100]    1
dtype: int64

Note the use of parentheses and square brackets. Consistent with mathematical notation for intervals, a parenthesis means that the side is *open* (exclude), while the square brackets means it is *closed* (inclusive). You can change which side is closed by passing `right=False`:

In [38]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [36, 61), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 11
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

Alternatively, you can pass your own bin names by passing a list of array to the `labels` option:

In [39]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels = group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., MiddleAged, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 11
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

Instead of passing in explicit bin edges, you can also pass in an integer number of bins to cut. By doing so, `pandas` will compute equal-length bins based on the minimum and maximum values in the data. Consider the case of some uniformly distributed data chopped into fourths:

In [40]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.025, 0.26], (0.5, 0.74], (0.74, 0.98], (0.025, 0.26], (0.5, 0.74], ..., (0.025, 0.26], (0.26, 0.5], (0.025, 0.26], (0.26, 0.5], (0.74, 0.98]]
Length: 20
Categories (4, interval[float64]): [(0.025, 0.26] < (0.26, 0.5] < (0.5, 0.74] < (0.74, 0.98]]

The `precision=2` option limits the decimal precision to two digits.

A closely related function, `qcut`, bins the data based on sample quantiles. Depending on the distribution of the data, using `cut` will not usually result in each bin having the same number of data points. Since `qcut` uses sample quantiles instead, by definition, you will obtain roughly equal-sized bins:

In [41]:
data = np.random.randn(1000)  # normally distributed

cats = pd.cut(data, 4)  # cut based on value intervals derived from maximum and minimum values
print(cats)
pd.value_counts(cats)   # this will not result in equal-sized bins

[(-1.188, 0.404], (0.404, 1.997], (1.997, 3.589], (0.404, 1.997], (-1.188, 0.404], ..., (0.404, 1.997], (-1.188, 0.404], (-1.188, 0.404], (-1.188, 0.404], (0.404, 1.997]]
Length: 1000
Categories (4, interval[float64]): [(-2.786, -1.188] < (-1.188, 0.404] < (0.404, 1.997] < (1.997, 3.589]]


(-1.188, 0.404]     515
(0.404, 1.997]      340
(-2.786, -1.188]    119
(1.997, 3.589]       26
dtype: int64

In [42]:
cats = pd.qcut(data, 4, precision=2)  # cut data distribution into quartiles
print(cats)
pd.value_counts(cats)

[(-0.71, 0.02], (0.68, 3.59], (0.68, 3.59], (0.02, 0.68], (-0.71, 0.02], ..., (0.68, 3.59], (0.02, 0.68], (-0.71, 0.02], (-0.71, 0.02], (0.02, 0.68]]
Length: 1000
Categories (4, interval[float64]): [(-2.7899999999999996, -0.71] < (-0.71, 0.02] < (0.02, 0.68] < (0.68, 3.59]]


(0.68, 3.59]                    250
(0.02, 0.68]                    250
(-0.71, 0.02]                   250
(-2.7899999999999996, -0.71]    250
dtype: int64

Similar to `cut`, you can pass your own quantiles (numbers between 0 and 1, inclusive):

In [43]:
cats = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1])
print(cats)
pd.value_counts(cats)

[(-1.273, 0.0197], (1.371, 3.589], (1.371, 3.589], (0.0197, 1.371], (-1.273, 0.0197], ..., (0.0197, 1.371], (0.0197, 1.371], (-1.273, 0.0197], (-1.273, 0.0197], (0.0197, 1.371]]
Length: 1000
Categories (4, interval[float64]): [(-2.7809999999999997, -1.273] < (-1.273, 0.0197] < (0.0197, 1.371] < (1.371, 3.589]]


(0.0197, 1.371]                  400
(-1.273, 0.0197]                 400
(1.371, 3.589]                   100
(-2.7809999999999997, -1.273]    100
dtype: int64

### Detecting and Filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations. Consider a DataFrame with some normally distributed data:

In [44]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.026699,0.040110,0.041304,0.017663
std,1.046804,0.973146,1.025736,0.998546
min,-3.019640,-3.114089,-3.158306,-3.259316
25%,-0.653157,-0.586727,-0.675379,-0.679950
50%,-0.037568,0.048412,0.017430,0.008312
75%,0.710212,0.655367,0.720244,0.705016
max,4.020619,2.810825,3.303404,2.909569


Now suppose you wanted to find value in one of the columns exceeding 3 in absolute value:

In [45]:
col = data[2]   # taking only 3rd column
col[np.abs(col) > 3]

91     3.303404
205   -3.158306
576    3.188950
674   -3.034271
Name: 2, dtype: float64

To select all rows having a value exceeding 3 or -3, you can use the `any` method on a boolean DataFrame:

In [46]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
91,1.219170,-0.302466,3.303404,-0.218652
205,-2.510459,-0.543303,-3.158306,0.355876
272,3.690463,-3.102172,-0.596735,-0.315749
364,1.286181,0.514747,-0.891047,-3.049286
576,0.939149,-0.563391,3.188950,0.443206
611,3.536758,-0.159994,-0.586040,-0.370652
639,-3.019640,0.251754,0.657559,0.595677
674,-0.041673,0.229322,-3.034271,-1.162565
717,-0.849549,-3.114089,0.894707,0.639573
756,4.020619,-0.978025,-0.182549,0.585699


To cap outlier values outside the interval -3 to 3, you can use the `np.sign()` method. The `np.sign(data)` will produce 1 and -1 values based on whether the values in `data` are positive or negative:

In [47]:
np.sign(data).head()

,0,1,2,3
0,-1.0,1.0,-1.0,-1.0
1,-1.0,1.0,-1.0,1.0
2,1.0,1.0,-1.0,-1.0
3,1.0,-1.0,-1.0,-1.0
4,1.0,-1.0,-1.0,1.0


In [48]:
data[np.abs(data) > 3] = np.sign(data) * 3  # capping values beyond 3 and -3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.024471,0.040326,0.041005,0.017971
std,1.039464,0.972457,1.023666,0.997577
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.653157,-0.586727,-0.675379,-0.679950
50%,-0.037568,0.048412,0.017430,0.008312
75%,0.710212,0.655367,0.720244,0.705016
max,3.000000,2.810825,3.000000,2.909569


### Permutation and Random Sampling

Permuting (randomly reordering) a Series or the rows in a DataFrame can be easily done using the `numpy.random.permutation` function. Calling `permutation` with the length of the axis you want to permute produces an array of integers indicating the new ordering: 

In [49]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))  # generating new DataFrame
sampler = np.random.permutation(5)  # randomise ordering in first 5 rows
sampler

array([0, 2, 3, 1, 4])

That array can then be used in iloc-based indexing or the equivalent `take` function:

In [50]:
print(df)

df.take(sampler)

    0   1   2   3
0   0   1   2   3
1   4   5   6   7
2   8   9  10  11
3  12  13  14  15
4  16  17  18  19


,0,1,2,3
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
1,4,5,6,7
4,16,17,18,19


To select a random subset without replacement, you can use the `sample` method on Series and DataFrame:

In [51]:
df.sample(n=3)

,0,1,2,3
2,8,9,10,11
1,4,5,6,7
0,0,1,2,3


For generating a sample _with_ replacement (i.e. to allow repeat choices), pass `replace=True` to `sample`:

In [52]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

3    6
2   -1
4    4
4    4
3    6
0    5
0    5
3    6
0    5
4    4
dtype: int64

### Computing Indicator/Dummy Variables

Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a "dummy" or "indicator" matrix. If a column in a DataFrame has `k` distinct values, you would derive a matrix or DataFrame with `k` columns containing all 1s and 0s. `pandas` has a `get_dummies` function for doing this, though devising one yourself is not difficult. Consider this example:

In [53]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})

pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In some cases, you may want to add a prefix to the columns in the indicator DataFrame, which can then be merged with the other data. `get_dummies` has a prefix argument for doing this:

In [54]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


If a row in a DataFrame belongs to multiple categories, things are a bit more complicated. Let\'s look at the MovieLens 1M dataset:

In [55]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::', header=None, names = mnames)
movies[:10]

<ipython-input-55-f33144af72f0>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table('datasets/movielens/movies.dat', sep='::', header=None, names = mnames)


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Adding indicator variables for each genre requires a little bit of wrangling. First, we extract the list of unique genres in the dataset:

In [56]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))  # extend list, append elements from the iterable

genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

One way to construct the indicator DataFrame is to start with a DataFrame of all zeroes. After which we have to iterate through each movie and set entries in each row of dummies to 1. To do this, we use the `dummies.columns` to compute the column indices for each genre:

In [57]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [58]:
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [59]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2])

Then, we can use `.iloc` to set values based on these indices:

In [60]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [61]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

_**Note**: For much larger data, this method of constructing indicator variables with multiple membership is not especially speedy. It is better to write a lower-level function that writes directly to a NumPy array, and then wrap the result in a DataFrame._

A useful recipe for statistical applications is to combine `get_dummies` witha discretisation function like `cut`:

In [62]:
values = np.random.rand(10)

bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,0,1,0,0,0
5,0,0,0,0,1
6,0,1,0,0,0
7,0,0,1,0,0
8,0,0,0,1,0
9,0,0,0,0,1


## String Manipulation 

Python has long been a popular raw data manipulation language in part due to its ease of use for string and text processing. Most text operations are made simple with the string object's built-in methods. For more complex pattern matching and text manipulations, regular expressions may be needed. `pandas` adds to the mix by enabling you to apply string and regular expressions concisely on whole array of data, additionally handling the annoyance of missing data.

### String Object Methods

 In many string munging and scripting applications, built-in string methods are sufficient. As an example, a comma-separated string can be broken into pieces with split:

In [63]:
val = 'a,b, guido'
val.split(',')

['a', 'b', ' guido']

`split` is often combined with `strip` to trim whitespaces (including line breaks):

In [64]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

These substrings can be concatenated together with a two-colon delimiter using addition:

In [65]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

A faster and more Pythonic way to concatenate in the above example is to pass a list or tuple to the `join` method on the string '::':

In [66]:
'::'.join(pieces)

'a::b::guido'

Other methods are concerned with locating substrings. Using Python's `in` keyword is best way to detect a substring, though `index` and `find` can also be used:

In [67]:
print('{}'.format('guido' in val))
print('{}'.format(val.index(',')))
print('{}'.format(val.find(':')))

True

1

-1


Table below shows a listing of some of Python's string methods

**Method** | **Description**
--- | ---
`count` | Return the number of non-overlapping occurences of substring in the string
`endswith` | Returns `True` if string ends with suffix
`startswith` | Returns `True` if string starts with prefix
`join` | Use string as delimiter for concatenating a sequence of other strings
`index` | Return position of first character in substring if found in the string; raises `ValueError` if not found.
`find` | Return position of first character of _first_ occurence of substring in the string; like `Index` but returns -1 if not found
`rfind` | Returns position of first character of _last_ occurence of substring in the string; returns -1 if not found.
`replace` | Replace occurences of string with another string
`strip, rstrip, lstrip` | Trim whitespace, including newlines; equivalent to `x.strip()` (and `rstrip` and `lstrip`, respectively
`split` | Break string into list of substrings using passed delimiter.
`lower` | Convert alphabet characters to lowercase
`upper` | Convert alphabet characters to uppercase
`casefold` | Convert characters to lowercase, and convert any region-specific variable character combinations to a common comparable form.
`ljust, rjust` | Left justify or right justify, respectively; pad opposite side of string with spaces (or some other fill character) to return a string with a minimum width

** Regular Expressions**